In [2]:
import xml.dom.minidom
import os
from shutil import copy2
import json
import pickle

# INITIALIZE DATASET PATTERNe
dataset = 'attgan'

# with open('captions_train2014.json', 'wb') as train_captions, \
#     open('captions_val2014.json', 'wb') as val_captions, \
#     open('imgs_train_path.txt', 'wb') as train_path, \
#     open('imgs_val_path.txt', 'wb') as val_path:   
with open(os.path.join('/media/hd1/home/wangwei/attn-gan/data/lung/train', \
                       'filenames.pickle'), 'wb') as train_names, \
     open(os.path.join('/media/hd1/home/wangwei/attn-gan/data/lung/test', \
                       'filenames.pickle'), 'wb') as test_names:
        
    if dataset in ('coco', 'genome'):
        train_json_data = {}
        train_json_data['images'] = []
        train_json_data['annotations'] = []        
        val_json_data = {}
        val_json_data['images'] = []
        val_json_data['annotations'] = []
        
        # we also need the following
        #res.dataset['info']
        #res.dataset['type']
        #res.dataset['licenses']
    elif dataset == 'karp':
        # init dict data
        json_data = {}
        json_data['images'] = []
        json_data['dataset'] = 'coco'
        
    elif dataset == 'attgan':
        train_name_list = []
        test_name_list = []
    
    else:
        print 'not implemented yet'
        
    
    i, j = 0, 0
    
    for xmlfile in os.listdir('../lung-report/ecgen-radiology/'): 
        print xmlfile

        DomTree = xml.dom.minidom.parse('../lung-report/ecgen-radiology/'+xmlfile)
        eCitation = DomTree.childNodes[0]
        parentImages = eCitation.getElementsByTagName('parentImage')

        MedlineCitation = eCitation.getElementsByTagName('MedlineCitation')[0]
        Abstract = MedlineCitation.getElementsByTagName('Abstract')[0]

        
        try:
            # COMPARISON = Abstract.getElementsByTagName('AbstractText')[0].firstChild.data
            # INDICATION = Abstract.getElementsByTagName('AbstractText')[1].firstChild.data
            FINDINGS = Abstract.getElementsByTagName('AbstractText')[2].firstChild.data
            # IMPRESSION = Abstract.getElementsByTagName('AbstractText')[3].firstChild.data
            
            # TRUNCATE LONG DESCRIPTION
            if len(FINDINGS.split(' ')) > 50:
                FINDINGS = ' '.join(FINDINGS.split(' ')[:50])
            
            j += 1 # report id

            for parentImage in parentImages:
                
                i += 1 # image id
                
                if dataset == 'coco':
                    anno = {}
                    anno['image_id'] = i
                    anno['id'] = j
                    anno['caption'] = FINDINGS
                    
                    img = {}
                    img['file_name'] = parentImage.attributes.items()[0][1] + '.png'
                    img['id'] = i
                    
                elif dataset == 'karp':
                    sent = {}
                    sent['imgid'], sent['sentids'] = i, i
                    sent['raw'] = FINDINGS
                    for char in  ['!', '?', '\'', ',', ';', '.', ':']:
                        FINDINGS = FINDINGS.replace(char, '')
                    sent['tokens'] = [x.lower() for x in FINDINGS.split(' ')]
                    
                    img = {}
                    img['filename'] = parentImage.attributes.items()[0][1] + '.png'
                    img['imgid'], img['sentids'], img['cocoid'] = i, [i], i
                    img['sentences'] = [sent]                    
                    
                elif dataset == 'attgan':
                    # one txt file for one caption
                    for char in  ['!', '?', '\'', ',', ';', '.', ':']:
                        FINDINGS = FINDINGS.replace(char, '')
                    with open(os.path.join('/media/hd1/home/wangwei/attn-gan/data/lung/text', \
                            (parentImage.attributes.items()[0][1] + '.txt')), 'wb') as txt_file:
                        txt_file.write(FINDINGS)
                
                # 70% 4532, 80% 5178, 90% 5825
                if i < 5825:  
                    if dataset == 'coco':
                        train_json_data['images'].append(img)
                        train_json_data['annotations'].append(anno)
                        
                    elif dataset == 'genome':
                        train_path.write('../lung-report/image/' + \
                                         parentImage.attributes.items()[0][1] + '.png' + '\n')                        
                        train_captions.write('{' + '\"image_id\": ' + \
                                             parentImage.attributes.items()[0][1] + ',' + \
                                             '\"id\": ' + str(j) + ',' + '\"paragraph\": ' + '\"' + \
                                             FINDINGS + '\"' + '},')
                        
                    elif dataset == 'karp': 
                        # sentence level, bottom up filling
                        img['split'] = 'train'
                        img['filepath'] = "train2014"
                        json_data['images'].append(img)
                        
                        copy2('../lung-report/image/'+ parentImage.attributes.items()[0][1] + '.png', \
                              './image/train2014/')
                              
                    elif dataset == 'attgan':
                        train_name_list.append(parentImage.attributes.items()[0][1])
                    
                else:                    
                    if dataset == 'coco':
                        val_json_data['images'].append(img)
                        val_json_data['annotations'].append(anno)
                                                
                    elif dataset == 'genome':
                        val_path.write('../lung-report/image/' + \
                                       parentImage.attributes.items()[0][1] + '.png' + '\n')
                        val_captions.write('{' + '\"image_id\": ' + \
                                           parentImage.attributes.items()[0][1] + ',' + \
                                           '\"id\": ' + str(j) + ',' + '\"paragraph\": ' + '\"' + \
                                           FINDINGS + '\"' + '},')

                    elif dataset == 'karp':                        
                        img['split'] = 'test' # also use for val
                        img['filepath'] = "test2014"
                        json_data['images'].append(img)
                        
                        # duplicate for val
                        img['split'] = 'val'
                        img['filepath'] = "val2014"
                        json_data['images'].append(img)
                        
                        copy2('../lung-report/image/'+ parentImage.attributes.items()[0][1] + '.png', \
                              './image/val2014/')
                              
                    elif dataset == 'attgan':
                        test_name_list.append(parentImage.attributes.items()[0][1])
                    
                print parentImage.attributes.items()[0][1] + '.png'
        
        except AttributeError:
            print 'This xml has no findings'
            pass

        
    if dataset in ('coco', 'genome'):
        json.dump(train_json_data, train_captions)
        json.dump(val_json_data, val_captions)
                              
    elif dataset == 'karp':
        json.dump(json_data, train_captions)
                              
    elif dataset == 'attgan':
        pickle.dump(train_name_list, train_names)
        pickle.dump(test_name_list, test_names)
        

In [4]:
%mv captions_train2014.json ./data/dataset_coco.json 
%rm imgs*.txt
%rm imgname*.json

mv: cannot stat ‘captions_train2014.json’: No such file or directory
